In [14]:
# base_path = './'

In [15]:
import pandas as pd
from time import time
import tensorflow as tf
import os

In [16]:
def gerar_rede(num_classes: int, n_neurons: int, n_hidden_layers: int):
    dir_path = os.getcwd()
    
    metrics_file = f"{dir_path}/training_metrics.csv"
    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Model Name", "Training Time", "Train Accuracy", "Test Accuracy"])

    file_data_train = os.path.join(dir_path, "iris_train.csv")
    file_data_test = os.path.join(dir_path, "iris_test.csv")
    
    data_train = pd.read_csv(file_data_train).to_numpy()
    data_test = pd.read_csv(file_data_test).to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    extentions = ['h5', 'keras']
    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons"
    )
    if not os.path.join(dir_path, "models"):
        os.makedirs(os.path.join(dir_path, "models"))
    

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        f'{model_path}.{extentions[1]}', monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    end = time() 
    
    print(f"Tempo de Treinamento: {end-start}")

    # salvar modelo
    model = tf.keras.models.load_model(f'{model_path}.{extentions[1]}')

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    # model.evaluate(x_test, y_test_ohe, verbose=2)
    model.save(f'{model_path}.{extentions[0]}')
    
    train_loss, train_acc = model.evaluate(x_train, y_train_ohe, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test_ohe, verbose=2)
    
    metrics_df.loc[len(metrics_df)] = { # type: ignore
        "Model Name": f"model_{n_hidden_layers}layers_{n_neurons}neurons",
        "Training Time": end - start,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc
    } 
    metrics_df.to_csv(metrics_file, index=False)

In [17]:
n_hidden_layers = 3
n_neurons = 20
num_classes = 3
gerar_rede(num_classes, n_neurons, n_hidden_layers)

Epoch 1/100
30/30 - 3s - 97ms/step - accuracy: 0.3250 - loss: 1.0369 - val_accuracy: 0.6000 - val_loss: 0.9969
Epoch 2/100
30/30 - 0s - 8ms/step - accuracy: 0.5583 - loss: 0.9701 - val_accuracy: 0.7000 - val_loss: 0.9211
Epoch 3/100
30/30 - 0s - 10ms/step - accuracy: 0.6333 - loss: 0.8972 - val_accuracy: 0.7000 - val_loss: 0.8349
Epoch 4/100
30/30 - 0s - 9ms/step - accuracy: 0.6583 - loss: 0.8090 - val_accuracy: 0.7000 - val_loss: 0.7338
Epoch 5/100
30/30 - 0s - 15ms/step - accuracy: 0.6583 - loss: 0.7130 - val_accuracy: 0.7333 - val_loss: 0.6225
Epoch 6/100
30/30 - 0s - 8ms/step - accuracy: 0.6833 - loss: 0.6168 - val_accuracy: 0.7667 - val_loss: 0.5202
Epoch 7/100
30/30 - 0s - 7ms/step - accuracy: 0.8250 - loss: 0.5086 - val_accuracy: 0.9000 - val_loss: 0.4327
Epoch 8/100
30/30 - 0s - 16ms/step - accuracy: 0.9333 - loss: 0.4345 - val_accuracy: 0.8000 - val_loss: 0.3674
Epoch 9/100
30/30 - 0s - 10ms/step - accuracy: 0.8583 - loss: 0.3856 - val_accuracy: 0.8667 - val_loss: 0.3607
Epoch

Resultado Teste
4/4 - 0s - 15ms/step - accuracy: 0.9667 - loss: 0.0820
1/1 - 0s - 54ms/step - accuracy: 0.9667 - loss: 0.1560
